import packages

In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import DataParallel
from torch.utils.data.sampler import RandomSampler, SequentialSampler
import torch.backends.cudnn as cudnn
import numpy as np

In [2]:
import cv2
import pandas as pd
from copy import deepcopy
from tqdm import tqdm
from importlib import import_module

import os
opj = os.path.join
ope = os.path.exists

In [3]:
from bestfitting.protein_clean.src.net import _init_paths
from bestfitting.protein_clean.src.net import densenet
from bestfitting.protein_clean.src.config.config import *
from bestfitting.protein_clean.src.dataset import protein_dataset
from bestfitting.protein_clean.src import train_net_base
from bestfitting.protein_clean.src import train_cls_net
from bestfitting.protein_clean.src.utils.augment_util import *
from bestfitting.protein_clean.src.net.loss_funcs.kaggle_metric import prob_to_result
from net.tool import *

run on ip-172-31-62-205
run on ip-172-31-62-205


external data (HPAv18)

In [5]:
header = 'external'
fname = opj(DATA_DIR, 'meta/%s_meta.csv'%(header))
df = pd.read_csv(fname)
print(fname,':', df.values.shape)

# image ids for external dataset 
external_id = df['Id'].values
external_id = np.sort(external_id)

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/meta/external_meta.csv : (74606, 33)


In [22]:
# image directory
external_orig_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18/jpg'
external_orig2_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18_2/jpg'
external_resize_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/external_v18_1536'

# load file names
external_orig = np.sort(os.listdir(external_orig_img_dir))
external_orig2 = np.sort(os.listdir(external_orig2_img_dir))
external_resize = np.sort(os.listdir(external_resize_img_dir))

In [23]:
# find images which are not in external_reize 
idx = np.argwhere(~np.isin(external_orig, external_resize)).ravel()
wrong_fnames = external_orig[idx]
print(len(wrong_fnames))

40


In [16]:
# check if wrong files in external_orig are correct in external_orig2
num = len(wrong_fnames)
none_list = np.empty(num, dtype=bool)
none_checking = np.empty(num, dtype=bool)

for i in range(num):
    fname = wrong_fnames[i]
    color = fname.replace('.jpg','').split('_')[-1]
    im = cv2.imread(opj(external_orig_img_dir, fname))
    if im is None:
        none_list[i] = True
    else:
        none_list[i] = False
        
    im_checking = cv2.imread(opj(external_orig2_img_dir, fname))
    if im_checking is None:
        none_checking[i] = True
    else:
        none_checking[i] = False
print(none_checking)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]


In [21]:
import shutil
# remove wrong files in external_orig and move from external_orig2
for i in range(num):
    fname = wrong_fnames[i]
    os.remove(opj(external_orig_img_dir, fname))
    shutil.move(opj(external_orig2_img_dir, fname), opj(external_orig_img_dir, fname))

    ### rerun resize code after moving files ###


### Check the number of images and labels for each dataset

In [26]:
# image directory
external_orig_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18/jpg'
external_orig2_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18_2/jpg'
external_resize_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/external_v18_1536'

# load file names
external_orig = np.sort(os.listdir(external_orig_img_dir))
external_orig2 = np.sort(os.listdir(external_orig2_img_dir))
external_resize = np.sort(os.listdir(external_resize_img_dir))

In [27]:
len(external_orig)

298424

In [28]:
len(external_orig2)

298384

In [30]:
len(external_resize)/4

74606.0

In [ ]:
print(np.sort(os.listdir(img_dir)).shape[0]/4)
print(np.sort(os.listdir(external_img_dir)).shape[0]/4)
print(np.sort(os.listdir(test_img_dir)).shape[0]/4)

In [ ]:
img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/tif/train/tifs'
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18/jpg'
test_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/tif/test/tifs'

In [ ]:
print(np.sort(os.listdir(img_dir)).shape[0]/4)
# print(np.sort(os.listdir(external_img_dir)).shape[0]/4)
print(np.sort(os.listdir(external_img_dir)).shape[0])
print(np.sort(os.listdir(test_img_dir)).shape[0]/4)


In [ ]:
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18/jpg'
file_list = np.sort(os.listdir(external_img_dir))

header = 'external'
fname = opj(DATA_DIR, 'meta/%s_meta.csv'%(header))
df = pd.read_csv(fname)
external_id = df['Id'].values
external_id = np.sort(external_id)

In [ ]:
external_list = external_id.copy()
color_list = ['_blue.jpg', '_green.jpg', '_red.jpg', '_yellow.jpg']
img_id = None
n_iter = 0

for i in range(len(file_list)):
    idx = i%4
    char = color_list[idx]
    img_id0 = img_id
    img_id = file_list[i].replace(char, '')

    if img_id != img_id0:
        external_list[n_iter] = img_id
        n_iter += 1

external_list = np.sort(external_list)

In [ ]:
np.sum(external_list == external_id) / len(external_id)

In [ ]:
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/external_v18_1536'
file_list = np.sort(os.listdir(external_img_dir))
len(file_list)/4

In [ ]:
file_list[8720:8730]

In [ ]:
# for i in range(len(file_list)):
#     idx = i%4
#     char = color_list[idx]
#     if file_list[i].find(char) == -1:
#         print('nope', i)

In [ ]:
char

In [ ]:
file_list[0].find(char)

In [ ]:
file_list[0].find('_green.jpg')

In [ ]:
external_resize_list = external_id.copy()[:74596]
color_list = ['_blue.jpg', '_green.jpg', '_red.jpg', '_yellow.jpg']
img_id = None
n_iter = 0

for i in range(len(file_list)):
    idx = i%4
    char = color_list[idx]
    img_id0 = img_id
    img_id = file_list[i].replace(char, '')

    if img_id != img_id0:
        external_resize_list[n_iter] = img_id
        n_iter += 1

external_resize_list = np.sort(external_resize_list)

In [ ]:
file_list[50:100]

In [ ]:
i

In [ ]:
len(file_list)

In [ ]:
np.sum(external_list == external_id) / len(external_id)

In [ ]:
header = 'train'
fname = opj(DATA_DIR, 'meta/%s_meta.csv'%(header))
df = pd.read_csv(fname)
print(fname,':', df.values.shape)

In [ ]:
fname = opj(DATA_DIR, 'split/train_31072.csv')
df = pd.read_csv(fname)
print(fname,':', df.values.shape)

In [ ]:
fname = opj(DATA_DIR, 'split/test_11702.csv')
df = pd.read_csv(fname)
print(fname,':', df.values.shape)

In [ ]:
split_alias = 'random_folds5'
num = []
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_train_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape)
    num.append(df.values.shape[0])
    
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_valid_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape, ';sum:', num[fold] + df.values.shape[0])    

In [ ]:
split_alias = 'random_ext_folds5'
num = []
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_train_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape)
    num.append(df.values.shape[0])
    
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_valid_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape, ';sum:', num[fold] + df.values.shape[0])    

In [ ]:
split_alias = 'random_ext_noleak_clean_folds5'
num = []
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_train_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape)
    num.append(df.values.shape[0])
    
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_valid_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape, ';sum:', num[fold] + df.values.shape[0])    

In [ ]:
img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/images_1536'
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/external_v18_1536'
test_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/test/images_1536'

In [ ]:
print(np.sort(os.listdir(img_dir)).shape[0]/4)
print(np.sort(os.listdir(external_img_dir)).shape[0]/4)
print(np.sort(os.listdir(test_img_dir)).shape[0]/4)

In [ ]:
img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/tif/train/tifs'
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18/jpg'
test_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/tif/test/tifs'

In [ ]:
print(np.sort(os.listdir(img_dir)).shape[0]/4)
# print(np.sort(os.listdir(external_img_dir)).shape[0]/4)
print(np.sort(os.listdir(external_img_dir)).shape[0])
print(np.sort(os.listdir(test_img_dir)).shape[0]/4)


In [ ]:
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18/jpg'
file_list = np.sort(os.listdir(external_img_dir))

header = 'external'
fname = opj(DATA_DIR, 'meta/%s_meta.csv'%(header))
df = pd.read_csv(fname)
external_id = df['Id'].values
external_id = np.sort(external_id)

In [ ]:
external_list = external_id.copy()
color_list = ['_blue.jpg', '_green.jpg', '_red.jpg', '_yellow.jpg']
img_id = None
n_iter = 0

for i in range(len(file_list)):
    idx = i%4
    char = color_list[idx]
    img_id0 = img_id
    img_id = file_list[i].replace(char, '')

    if img_id != img_id0:
        external_list[n_iter] = img_id
        n_iter += 1

external_list = np.sort(external_list)

In [ ]:
np.sum(external_list == external_id) / len(external_id)

In [ ]:
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/external_v18_1536'
file_list = np.sort(os.listdir(external_img_dir))
len(file_list)/4

In [ ]:
file_list[8720:8730]

In [ ]:
# for i in range(len(file_list)):
#     idx = i%4
#     char = color_list[idx]
#     if file_list[i].find(char) == -1:
#         print('nope', i)

In [ ]:
char

In [ ]:
file_list[0].find(char)

In [ ]:
file_list[0].find('_green.jpg')

In [ ]:
external_resize_list = external_id.copy()[:74596]
color_list = ['_blue.jpg', '_green.jpg', '_red.jpg', '_yellow.jpg']
img_id = None
n_iter = 0

for i in range(len(file_list)):
    idx = i%4
    char = color_list[idx]
    img_id0 = img_id
    img_id = file_list[i].replace(char, '')

    if img_id != img_id0:
        external_resize_list[n_iter] = img_id
        n_iter += 1

external_resize_list = np.sort(external_resize_list)

In [ ]:
file_list[50:100]

In [ ]:
i

In [ ]:
len(file_list)

In [ ]:
np.sum(external_list == external_id) / len(external_id)